# 10주 기념축사 TF-IDF
'제74주년 경찰의 날 기념식 축사' 전문을 읽고 TF-IDF를 계산하세요.

전문은 http://www.korea.kr/archive/speechView.do?newsId=132031636 에서 읽을 수 있고,

해당 사이트에서 텍스트만 파일로 저장한 후 사용한다.



* 1) DataFrame을 생성

* 2) 단어로 분리해서, 출력

* 3) 불용어 구성, 출력 - 축사 전문에서 한글자로 된 단어를 찾아내 스스로 구성

* 4) 불용어 제거하고, 출력

* 5) TF-IDF를 계산하고, 출력

* 6) TF-IDF 컬럼을 features로 구성, 출력



* 제출: ipynb 파일 1개 (zip하지 마세요).

* 기한: 2021/11/09 화 22:00시

# 1) DataFrame을 생성

In [6]:
import pyspark

sparkConfig = pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=sparkConfig)\
    .getOrCreate()

In [9]:
import os
from pyspark.sql.types import StructType, StructField, StringType

police=spark.read\
    .options(header="true", delimiter=" ", inferSchema="true")\
    .schema(
        StructType([
            StructField("sent",StringType()),
            ])
    )\
    .text(os.path.join("data", "20211109_policeAddress.txt"))

In [10]:
police.show()

+----------------------------------+
|                              sent|
+----------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|
|                                  |
| 국민의 안전을 위해 밤낮없이 애...|
|                                  |
|오늘 홍조근정훈장을 받으신 중앙...|
|                                  |
|           사랑하는 경찰관 여러분,|
|                                  |
|여러분의 헌신적 노력으로 우리의...|
|                                  |
| 치안의 개선은 국민의 체감으로 ...|
|                                  |
| 한국을 찾는 외국 관광객들도 우...|
|                                  |
|   올해는 ‘경찰의 날’에 맞춰 국...|
|                                  |
|         자랑스러운 경찰관 여러분,|
|                                  |
| 경찰헌장은 “나라와 겨레를 위하...|
|                                  |
+----------------------------------+
only showing top 20 rows



# 2) 단어로 분리해서, 출력

In [11]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="sent", outputCol="tokens")
tokDf = tokenizer.transform(police)

In [12]:
tokDf.show(3)

+---------------------------------+------------------------------+
|                             sent|                        tokens|
+---------------------------------+------------------------------+
| 존경하는 국민 여러분, 경찰관 ...| [존경하는, 국민, 여러분,, ...|
|                                 |                            []|
|국민의 안전을 위해 밤낮없이 애...|[국민의, 안전을, 위해, 밤낮...|
+---------------------------------+------------------------------+
only showing top 3 rows



# 3) 불용어 구성, 출력 - 축사 전문에서 한글자로 된 단어를 찾아내 스스로 구성

In [46]:
import re
from pyspark.sql import functions as f
from pyspark.sql.types import ArrayType, StringType

def trim(wordList):
    regex = re.compile('\d+')
    cleaned=list()
    for w in wordList:
        if not regex.match(w):
            cleaned.append(w.lstrip('‘').rstrip("’").rstrip(',').rstrip('.').replace("’","").replace("”",""))
    return cleaned

trimUdf=f.udf(trim, ArrayType(StringType()))

In [31]:
wordsDf = tokDf.withColumn('words', trimUdf(f.col('tokens')))
wordsDf.select("words").show(3, False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|words                                                                                                                                                                                            |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[존경하는, 국민, 여러분, 경찰관, 여러분, 일흔네, 돌, 경찰의, 날입니다]                                                                                                                           |
|[]                                                                                                                                                                                               |
|[국민의, 안전을, 위해, 밤낮없이, 애쓰시는, 전국의, 경찰관, 여러분께, 먼저

In [52]:
allWords = wordsDf.select('words').rdd.flatMap(lambda x: x).flatMap(lambda x: x).collect()

krStopwords = list(set(list(filter(lambda x: len(x) == 1, test))))
krStopwords

['참', '그', '법', '등', '돌', '더', '될', '큰', '수']

# 4) 불용어 제거하고, 출력

In [50]:
def filterStopword(wordList):
    krStopwords = ['참', '그', '법', '등', '돌', '더', '될', '큰', '수']
    cleaned = trim(wordList)
    cleaned = list(filter(lambda x: len(x) != 0, cleaned))
    cleaned = list(filter(lambda x: x not in krStopwords, cleaned))
    
    return cleaned

filterStopword(allWords)

['존경하는',
 '국민',
 '여러분',
 '경찰관',
 '여러분',
 '일흔네',
 '경찰의',
 '날입니다',
 '국민의',
 '안전을',
 '위해',
 '밤낮없이',
 '애쓰시는',
 '전국의',
 '경찰관',
 '여러분께',
 '먼저',
 '감사를',
 '드립니다',
 '전몰·순직',
 '경찰관들의',
 '고귀한',
 '희생에',
 '경의를',
 '표합니다',
 '유가족',
 '여러분께',
 '위로의',
 '마음을',
 '전합니다',
 '오늘',
 '홍조근정훈장을',
 '받으신',
 '중앙경찰학교장',
 '이은정',
 '치안감님',
 '근정포장을',
 '받으신',
 '광주남부경찰서',
 '김동현',
 '경감님을',
 '비롯한',
 '수상자',
 '여러분께',
 '각별한',
 '축하와',
 '감사를',
 '드립니다',
 '또한',
 '경찰',
 '영웅으로',
 '추서되신',
 '차일혁',
 '최중락님께',
 '국민의',
 '사랑을',
 '전해드립니다',
 '사랑하는',
 '경찰관',
 '여러분',
 '여러분의',
 '헌신적',
 '노력으로',
 '우리의',
 '치안은',
 '좋아졌습니다',
 '지난해',
 '범죄',
 '발생은',
 '비해',
 '줄었습니다',
 '같은',
 '기간',
 '교통사고',
 '사망자는',
 '감소했습니다',
 '치안의',
 '개선은',
 '국민의',
 '체감으로',
 '나타나고',
 '있습니다',
 '올해',
 '상반기',
 '국민의',
 '체감안전도는',
 '역대',
 '최고를',
 '기록했습니다',
 '범죄안전도는',
 '처음으로',
 '넘었습니다',
 '한국을',
 '찾는',
 '외국',
 '관광객들도',
 '우리의',
 '치안을',
 '가장',
 '좋게',
 '평가합니다',
 '한국의',
 '무엇이',
 '좋았느냐는',
 '물음에',
 '외국',
 '관광객들은',
 '연속으로',
 '치안이',
 '가장',
 '좋았다고',
 '응답했습니다',
 '개발도상국들은',
 '우리의',
 '경찰을',
 '모범으로',
 '삼으려'

In [53]:
from pyspark.ml.feature import StopWordsRemover

stop = StopWordsRemover(inputCol="words", outputCol="nostops")
stop.setStopWords(krStopwords)

StopWordsRemover_f28806a35344

In [57]:
stopDf=stop.transform(wordsDf)
stopDf.select("nostops").show()

+--------------------------------+
|                         nostops|
+--------------------------------+
|  [존경하는, 국민, 여러분, 경...|
|                              []|
|  [국민의, 안전을, 위해, 밤낮...|
|                              []|
|[오늘, 홍조근정훈장을, 받으신...|
|                              []|
|      [사랑하는, 경찰관, 여러분]|
|                              []|
| [여러분의, 헌신적, 노력으로,...|
|                              []|
|  [치안의, 개선은, 국민의, 체...|
|                              []|
|  [한국을, 찾는, 외국, 관광객...|
|                              []|
|  [올해는, 경찰의, 날에, 맞춰...|
|                              []|
|    [자랑스러운, 경찰관, 여러분]|
|                              []|
| [경찰헌장은, “나라와, 겨레를...|
|                              []|
+--------------------------------+
only showing top 20 rows



In [56]:
for r in stopDf.select("nostops").take(3):
    for e in r:
        print (e)

['존경하는', '국민', '여러분', '경찰관', '여러분', '일흔네', '경찰의', '날입니다']
['']
['국민의', '안전을', '위해', '밤낮없이', '애쓰시는', '전국의', '경찰관', '여러분께', '먼저', '감사를', '드립니다', '전몰·순직', '경찰관들의', '고귀한', '희생에', '경의를', '표합니다', '유가족', '여러분께', '위로의', '마음을', '전합니다']


# 5) TF-IDF를 계산하고, 출력

In [58]:
from pyspark.ml.feature import HashingTF, IDF

hashTF = HashingTF(inputCol="nostops", outputCol="hash")

hashDf = hashTF.transform(stopDf)

idf = IDF(inputCol="hash", outputCol="idfb")

idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)

idfDf.select("nostops", "hash").show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|nostops                                                                                                                                                                                                                                                             |hash                                                                                                                                                                                       

# 6) TF-IDF 컬럼을 features로 구성, 출력

In [59]:
from pyspark.ml.feature import Word2Vec

word2Vec = Word2Vec(vectorSize=20, minCount=1, inputCol="tokens", outputCol="w2v")
model = word2Vec.fit(tokDf)
w2vDf = model.transform(tokDf)
model.getVectors().show()

+------------------------+--------------------+
|                    word|              vector|
+------------------------+--------------------+
|                공정해야|[-0.0184104871004...|
|정책브리핑(www.korea.kr)|[0.02456313185393...|
|              자치경찰제|[0.01200542598962...|
|                공권력을|[0.00427275570109...|
|             흘렸습니다.|[-0.0149954678490...|
|                  만들고|[0.02153245359659...|
|                  헌신에|[0.00691718002781...|
|                 나창헌,|[0.00580025278031...|
|                    닿아|[-0.0188714303076...|
|                      돌|[0.00250513409264...|
|                        |[0.00835204124450...|
|                  민갑룡|[0.01696237549185...|
|                  수상자|[0.00560041284188...|
|                    기간|[0.01902047730982...|
|                  시대적|[0.01845362596213...|
|                  일흔네|[0.01772616431117...|
|               했습니다.|[0.00580584257841...|
|                    범죄|[-0.0220075789839...|
|               됐습니다.|[0.01361862849444...|
|  